# Tensor Report Card

Not all bloq examples support tensor simulation. This report card automatically determines which bloq examples should be tensor simulable.

 - State vector simulation uses $2^n$ numbers to simulate a quantum state. The tensor protocol uses quimb to try to find more efficient contraction orderings. Quimb reports the contraction width, which is the minimum size of any intermediate tensor encountered in the contraciton. The simulation uses $2^w$ numbers, where $w$ is the width. We consider a width under 25 qubits as simulable.
 - Qualtran requires "flattening" out the bloq to turn it into an efficient tensor network. This may take too much time itself for large algorithms with many levels of abstraction. If the process of turning a bloq into a quimb tensor network and finding a contraction ordering takes longer than 8 seconds, we don't consider the bloq simulable.
 - The flattened structure needs to have explicit tensors. For bloqs with symbolic parameters, we either can't decompose & flatten them, or the tensors would be symbolic, which we don't support.

In [ ]:
from qualtran_dev_tools.bloq_finder import get_bloq_examples
from qualtran_dev_tools.tensor_report_card import report_on_tensors, ExecuteWithTimeout

In [ ]:
bes = get_bloq_examples()

# Imports to exclude certain bloqs, see following comment
from qualtran.bloqs.multiplexers.apply_gate_to_lth_target import ApplyGateToLthQubit

In [ ]:
exec = ExecuteWithTimeout(timeout=8., max_workers=4)
for i, be in enumerate(bes):

    if be.bloq_cls == ApplyGateToLthQubit:
        # This bloq uses a lambda function as one of its attributes, which
        # can't be pickled and used with multiprocessing.
        continue
    
    exec.submit(report_on_tensors, kwargs=dict(name=be.name, cls_name=be.bloq_cls.__name__, bloq=be.make()))

records = []
while exec.work_to_be_done:
    kwargs, record = exec.next_result()
    #print(kwargs['name'], end=' ', flush=True)
    print('\r', f'{exec.work_to_be_done:5d} remaining', end='', flush=True)
    
    if record is None:
        records.append({
            'name': kwargs['name'],
            'cls': kwargs['cls_name'],
            'err': 'Timeout',
        })
    else:
        records.append(record)

import pandas as pd
df = pd.DataFrame(records)

## Number of bloq examples considered

In [ ]:
print(len(df))

## Number of bloq examples successfully flattened

In [ ]:
print(len(df[df['flat_dur'] > 0]))

## Number of bloq examples with tensors

In [ ]:
print(len(df[df['width'] > 0]))

## Bloqs that are tensor simulable

In [ ]:
print(len(df[df['width'] <= 25]))
df[df['width'] <= 25]

## Bloqs whose tensor network is too big

In [ ]:
df[df['width'] > 25].sort_values(by='width')

## Bloqs without tensors

Due to errors encountered in flattening or if the bloq's callees don't support tensor simulation.

In [ ]:
df[df['width'].isna()]

## Slowest to flatten

Within the overall timeout

In [ ]:
df.sort_values(by='flat_dur', ascending=False).head()

## Flattening is the rate-limiting step.

For bloqs that have been successfully flattened, the maximum tensor-network-construction and tensor-contraction-ordering durations are less than 0.5s. Note: the contraction finding code uses the fast, naive approach. One can choose more expensive approaches where the contraciton-ordering-finding is more expensive.

In [ ]:
# Slowest tn_dur
df.sort_values(by='tn_dur', ascending=False).head()

In [ ]:
# Slowest width_dur
df.sort_values(by='width_dur', ascending=False).head()